The purpose of this exercise is to extract new features from the categorical and numeric variables before the modeling phase. In the previous chaters, we applied various feature extraction techniques, such as converting categorical variables to dummy variables and scaling variables. This exercise will demonstrate how these task can be automated using ML Pipelines.

In [1]:
import pandas as pd
file_url = 'https://raw.githubusercontent.com/sedeba19/Chapter-16/main/data_source/Dataset_crx.data.txt'

df = pd.read_csv(file_url,
                 sep = ',',
                 header = None,
                 na_values= '?')

# Changing the Classess to 1 & 0
df.loc[df[15] == '+', 15] = 1
df.loc[df[15] == '-', 15] = 0

df_clean = df.dropna(axis = 0)
df_clean.isna().sum()

# Separating X and y variabls
X = df_clean.loc[:, 0:14]
y = df_clean.loc[:, 15].astype('int')

from sklearn.model_selection import train_test_split

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size= 0.3,
                                                    random_state=123)

Create Processing Engine

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Pipeline for transforming categorical variables
catTransformer = Pipeline(steps = [('onehot', OneHotEncoder(handle_unknown = 'ignore'))])
catTransformer

Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [3]:
# Pipeline for scaling numerical variables
numTransformer = Pipeline(steps = [('scaler', StandardScaler())])
numTransformer 

Pipeline(steps=[('scaler', StandardScaler())])

In [4]:
X.dtypes

0      object
1     float64
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12     object
13    float64
14      int64
dtype: object

In [5]:
catFeatures = X.select_dtypes(include = 'object').columns
catFeatures

Int64Index([0, 3, 4, 5, 6, 8, 9, 11, 12], dtype='int64')

In [6]:
numFeatures = X.select_dtypes(include = ['float', 'int']).columns
numFeatures

Int64Index([1, 2, 7, 10, 13, 14], dtype='int64')

Just to get the context of what we are going to do next, we are going to create a literal engine that automates the task of scaling features and converting categorical variables to a one-hot encoded form.

In [7]:
# Create the preprocessing engine
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[('numeric', numTransformer, numFeatures),
                                               ('categoric', catTransformer, catFeatures)])
preprocessor

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 Int64Index([1, 2, 7, 10, 13, 14], dtype='int64')),
                                ('categoric',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Int64Index([0, 3, 4, 5, 6, 8, 9, 11, 12], dtype='int64'))])

In [12]:
# Import libraries
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier

# Create a pipeline with AdaBoostClassifier model
ada_classf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('dimred', PCA()),
                      ('classifier', AdaBoostClassifier(random_state=123))])
ada_classf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Int64Index([1, 2, 7, 10, 13, 14], dtype='int64')),
                                                 ('categoric',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Int64Index([0, 3, 4, 5, 6, 8, 9, 11, 12], dtype='int64'))])),
                ('dimred', PCA()),
                ('classifier', AdaBoostClassifier(random_state=123))])

In [14]:
# Define the parameters as a dictionary for gridsearch
param_grid = {'dimred__n_components': [10, 12, 15],
              'classifier__n_estimators': [50, 100, 200],
              'classifier__learning_rate': [0.7, 0.6, 1.0]}

In [15]:
# Create an estimator that will be used to improve the parameter of a model
from sklearn.model_selection import GridSearchCV
estimator_gs = GridSearchCV(ada_classf,
                            cv = 10,
                            param_grid= param_grid)
estimator_gs

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Int64Index([1, 2, 7, 10, 13, 14], dtype='int64')),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Int64Index([0, 3, 4, 5, 6, 8, 9, 11, 12], dtype='int64'))])),
                                       ('dimred', PCA()),
                                       ('classifier',
                                        AdaBoostClassifier(random_state=123))]),
             param_grid={'classifier__learning_rate': [0.7, 0.6, 1.0],
                         'classifier__n_estimators': [50, 100, 200],
                         'dimred__n_components': [10, 12, 15]})

In [16]:
# Fit the data into the model
estimator_gs.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Int64Index([1, 2, 7, 10, 13, 14], dtype='int64')),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Int64Index([0, 3, 4, 5, 6, 8, 9, 11, 12], dtype='int64'))])),
                                       ('dimred', PCA()),
                                       ('classifier',
                                        AdaBoostClassifier(random_state=123))]),
             param_grid={'classifier__learning_rate': [0.7, 0.6, 1.0],
                         'classifier__n_estimators': [50, 100, 200],
                         'dimred__n_components': [10, 12, 15]})

In [22]:
# print the best score and best parameters of this model/ algorithm (AdaboostClassifier)
print('best: %f using %s' %(estimator_gs.best_score_, estimator_gs.best_params_))

best: 0.842464 using {'classifier__learning_rate': 0.7, 'classifier__n_estimators': 50, 'dimred__n_components': 15}


In [23]:
# Make predictions
pred = estimator_gs.predict(X_test)
pred

array([0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1])

In [24]:
len(pred)

196

In [26]:
# Print classification report
from sklearn.metrics import classification_report, confusion_matrix
conf_matrix = confusion_matrix(y_test, pred)
conf_matrix

array([[90, 17],
       [14, 75]], dtype=int64)

In [27]:
cl_report = classification_report(y_test, pred)
print(cl_report)

              precision    recall  f1-score   support

           0       0.87      0.84      0.85       107
           1       0.82      0.84      0.83        89

    accuracy                           0.84       196
   macro avg       0.84      0.84      0.84       196
weighted avg       0.84      0.84      0.84       196

